# Ensemble Model

Complete ensemble model implementation combining PyTorch NN and Gradient Boosting with probability calibration using StackingClassifier and CalibratedClassifierCV.

1. PyTorch neural network (best performing single model)
2. Gradient Boosting (different algorithm = diversity)
3. Stacking classifier with logistic regression meta-learner


In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

import numpy as np
import json
import pickle
from sklearn.ensemble import GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, precision_score, recall_score, brier_score_loss, log_loss

# Import PyTorch model class
from src.app.pytorch_classifier import PyTorchPatentClassifier

# Note: This notebook assumes PyTorch model is already trained and saved
# The PyTorch model class is defined in src/app/pytorch_classifier.py

: 

## Load Features


In [ ]:
features_dir = Path("data/features")

X_train = np.load(features_dir/"train_features_v2.X.npy")
y_train = np.load(features_dir/"train_features_v2.y.npy")
X_val = np.load(features_dir/"val_features_v2.X.npy")
y_val = np.load(features_dir/"val_features_v2.y.npy")
X_test = np.load(features_dir/"test_features_v2.X.npy")
y_test = np.load(features_dir/"test_features_v2.y.npy")

with open(features_dir/"feature_names_v2.json", "r") as f:
    feature_names = json.load(f)

# Removed BM25 and CPC features (indices 0, 1, 6) to match 10-feature models (after ablation study) 
indices_to_remove = [0, 1, 6]
indices_to_keep = [i for i in range(13) if i not in indices_to_remove]
X_train = X_train[:, indices_to_keep]
X_val = X_val[:, indices_to_keep]
X_test = X_test[:, indices_to_keep]

print(f"Training: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")

Training: (39979, 10), Validation: (8567, 10), Test: (8568, 10)


## Load PyTorch Model


In [6]:
# Load PyTorch model
pytorch_model = PyTorchPatentClassifier()
pytorch_model.load(Path('models/pytorch_nn'))

# Prepare scaled features for meta-learner
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

## Obtain PyTorch Model Predictions


In [ ]:
# Get PyTorch model predictions (probabilities for class 1)
pytorch_proba_train = pytorch_model.predict_proba(X_train)[:, 1]
pytorch_proba_val = pytorch_model.predict_proba(X_val)[:, 1]
pytorch_proba_test = pytorch_model.predict_proba(X_test)[:, 1]

# Create meta-features: PyTorch predictions + original scaled features
X_meta_train = np.column_stack([pytorch_proba_train, X_train_scaled])
X_meta_val = np.column_stack([pytorch_proba_val, X_val_scaled])
X_meta_test = np.column_stack([pytorch_proba_test, X_test_scaled])

print(f"Meta-features shape: {X_meta_train.shape}")
print(f"PyTorch predictions: 1 feature")
print(f"Original features: {X_train_scaled.shape[1]} features")

Meta-features shape: (39979, 11)
PyTorch predictions: 1 feature
Original features: 10 features


## Train Stacking Ensemble


In [10]:
# Base model: Gradient Boosting (trained on PyTorch predictions + original features)
# Meta-learner: Logistic Regression (learns to combine base model predictions)

# The meta-features (X_meta) include:
# 1. PyTorch model predictions (1 feature)
# 2. Original scaled features (10 features)

base_models = [
    ('gb', GradientBoostingClassifier(n_estimators=100, max_depth=5, random_state=42))
]

meta_model = LogisticRegression(random_state=42, max_iter=1000)

stacking = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5,
    n_jobs=-1
)

stacking.fit(X_meta_train, y_train)

## Calibrate Probabilities


In [12]:
calibrated_ensemble = CalibratedClassifierCV(
    stacking,
    method='sigmoid',
    cv=5 # 5-fold cross-validation
)

calibrated_ensemble.fit(X_meta_val, y_val)

## Evaluate Ensemble


In [14]:
y_pred_test = calibrated_ensemble.predict(X_meta_test)
y_proba_test = calibrated_ensemble.predict_proba(X_meta_test)[:, 1]

metrics = {
    'accuracy': accuracy_score(y_test, y_pred_test),
    'precision': precision_score(y_test, y_pred_test),
    'recall': recall_score(y_test, y_pred_test),
    'f1': f1_score(y_test, y_pred_test),
    'roc_auc': roc_auc_score(y_test, y_proba_test),
    'brier_score': brier_score_loss(y_test, y_proba_test),
    'log_loss': log_loss(y_test, y_proba_test)
}

print("Test Metrics:")
for key, value in metrics.items():
    print(f"  {key}: {value:.4f}")


Test Metrics:
  accuracy: 0.9158
  precision: 0.9221
  recall: 0.9067
  f1: 0.9143
  roc_auc: 0.9713
  brier_score: 0.0663
  log_loss: 0.2327


## Save Ensemble Model


In [16]:
ensemble_dir = Path("experimental/ensemble/models")
ensemble_dir.mkdir(parents=True, exist_ok=True)

with open(ensemble_dir/"ensemble_model.pkl", "wb") as f:
    pickle.dump(calibrated_ensemble, f)

with open(ensemble_dir/"scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)

results_dir = Path("experimental/ensemble/results")
results_dir.mkdir(parents=True, exist_ok=True)

with open(results_dir/"ensemble_metrics.json", "w") as f:
    json.dump(metrics, f, indent=2)

print(f"Ensemble model saved to: {ensemble_dir}")
print(f"Metrics saved to: {results_dir}")

Ensemble model saved to: experimental/ensemble/models
Metrics saved to: experimental/ensemble/results
